# RDF2MARCXML

Replace the URI with your Instance URI from Sinopia to generate MARCXML based on the Library of Congress BIBFRAME2MARC mapping.


In [1]:
!pip install rdflib
!pip install lxml
!pip install subprocess.run

In [2]:
import subprocess
from marc_xml.lc_bfxml_work import lc_bfxml_work, remove_last_line
from marc_xml.lc_bfxml_instance import lc_bfxml_instance, remove_rdf_header

#replace uri with the Instance uri of the resource you want to convert
uri = "https://api.development.sinopia.io/resource/90fb418e-4ff4-4296-af69-1e30e2ac2f8a"

lc_bfxml_work(uri)  
remove_last_line() 
lc_bfxml_instance(uri) 
remove_rdf_header() 


In [3]:
# combine the two files use work first then add the instance
with open("lc_bfxml_work.xml", "r") as work_file:
    work = work_file.read()
with open("lc_bfxml_instance.xml", "r") as instance_file:
    instance = instance_file.read()

#save as a file name LoC_Work_Instance.xml
with open("LoC_Work_Instance.xml", "w") as combined_file:
    combined_file.write(work + instance)

# add the sinopiabf namespace to the combined file
with open("LoC_Work_Instance.xml", "r") as file:
    filedata = file.read()
filedata = filedata.replace('<rdf:RDF', '<rdf:RDF xmlns:sinopiabf="http://sinopia.io/vocabulary/bf/"')
with open("LoC_Work_Instance.xml", "w") as file:
    file.write(filedata)

# apply "pre-transform-normalize.xsl" on LoC_Work_Instance.xml file for normalization
import lxml.etree as ET
dom = ET.parse("LoC_Work_Instance.xml")
xslt = ET.parse("marc_xml/xsl/pre-transform-normalize.xsl")
transform = ET.XSLT(xslt)
newdom = transform(dom)
with open("LoC_Work_Instance_Normalized.xml", "w") as f:
    f.write(str(newdom))    


# use xsltproc for BF2MARC conversion
subprocess.run(['xsltproc', '-o', 'marc.xml', '../bibframe2marc/bibframe2marc.xsl', 'LoC_Work_Instance_Normalized.xml'])

# print the marc.xml file
with open("marc.xml", "r") as file:
    print(file.read())


<?xml version="1.0" encoding="UTF-8"?>
<marc:record xmlns:marc="http://www.loc.gov/MARC21/slim">
  <marc:leader>     nam a22     uu 4500</marc:leader>
  <marc:controlfield tag="001">idp140538472774831</marc:controlfield>
  <marc:controlfield tag="003">DLC</marc:controlfield>
  <marc:controlfield tag="008">231019|||||||||xx a          00| 0 chi  </marc:controlfield>
  <marc:datafield tag="020" ind1=" " ind2=" ">
    <marc:subfield code="a">9789881902580</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="020" ind1=" " ind2=" ">
    <marc:subfield code="a">9881902584</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="035" ind1=" " ind2=" ">
    <marc:subfield code="a">1127933250</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="040" ind1=" " ind2=" ">
    <marc:subfield code="b">eng</marc:subfield>
    <marc:subfield code="d">DLC</marc:subfield>
    <marc:subfield code="e">rda</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="043" ind1=" " ind2=" ">
    <

Record idp140538472774831: Unprocessed node bf:language. Non-repeatable target element 008.
Record idp140538472774831: Unprocessed node bf:assigner. Non-repeatable target field (040).
